# Adicionando funções e ferramentas para o Assistent

In [1]:
import json

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

# Definindo função

In [2]:
def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    elif "rio de janeiro" in local.lower():
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )

# Definindo tools

#### Define as ferramentas que o modelo pode usar para gerar as respostas. Uma função que pode ser usada pelo modelo.
#### Formato específico de dicionários aninhados dentro de uma lista, para que o modelo entenda.

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]

# Mapenado funções

#### Mapeia nomes de funções aos objetos de função, para ser mais facil de chamar

In [4]:
funcoes_disponiveis = {
        "obter_temperatura_atual": obter_temperatura_atual,
    }

# Enviando mensagem e capturando resposta

#### O parâmetro tools  informa ao modelo quais ferramentas (funções) estão disponíveis para ele utilizar.  

####  Tool_choice="auto" = indica ao modelo qual ferramenta utilizar. Quando "auto", ele decide automaticamente quando e como usa-las.


In [5]:
mensagens = [
    {"role": "user", 
     "content": "Qual é a temperatura em São Paulo e Porto Alegre?"}
    ]


resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto",
)



## Como fica a variavel resposta

In [11]:
print(resposta.choices[0].message.content)

None


#### Podemos ver que ainda não há resposta... Mas ele tem algo pra nos informar, ele precisa de algo pra nos dar a resposta:

In [7]:
resposta

ChatCompletion(id='chatcmpl-9PvOpQGkQJmjCAgqheiMxHX1rbQIU', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_duaszJWi9ZZ6d14GZJcSqiV3', function=Function(arguments='{"local": "São Paulo", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_EtY9TgZ7HqLTYJoTJ0QtmJaZ', function=Function(arguments='{"local": "Porto Alegre", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')]))], created=1715965871, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=69, prompt_tokens=90, total_tokens=159))

### Pegando realmente so o que nos interessa da resposta

In [8]:
mensagem_resp = resposta.choices[0].message
mensagem_resp

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_duaszJWi9ZZ6d14GZJcSqiV3', function=Function(arguments='{"local": "São Paulo", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function'), ChatCompletionMessageToolCall(id='call_EtY9TgZ7HqLTYJoTJ0QtmJaZ', function=Function(arguments='{"local": "Porto Alegre", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')])

# Aplicando tool_calls

#### O `.tool_calls` extrai e lista chamadas de ferramentas que o modelo solicita durante a geração da resposta. A finalidade é identificar quais funções precisam ser executadas para fornecer informações  para a resposta do modelo.

Apresenta o id, a Function, seu nome e  seus argumentos. Podemos ver abaixo que houve 2 chamadas.

### Então para nos dar a resposta, nos precisamos rodar essas funções com os parametros identificados



In [9]:
tool_calls = mensagem_resp.tool_calls
tool_calls

[ChatCompletionMessageToolCall(id='call_duaszJWi9ZZ6d14GZJcSqiV3', function=Function(arguments='{"local": "São Paulo", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function'),
 ChatCompletionMessageToolCall(id='call_EtY9TgZ7HqLTYJoTJ0QtmJaZ', function=Function(arguments='{"local": "Porto Alegre", "unidade": "celsius"}', name='obter_temperatura_atual'), type='function')]

### E para cada chamada dessa, vamos chamar a função para o modelo conseguir as respostas:

In [10]:
if tool_calls:  # Se houver chamada de ferramentas
    mensagens.append(mensagem_resp)   # Adicione a resposta na conversa
    for tool_call in tool_calls:      # Para cada chamada
        function_name = tool_call.function.name  # Extraia o nome da função
        function_to_call = funcoes_disponiveis[function_name] # Obtem a função, atraves de 'funcoes_disponiveis' e seu nome
        function_args = json.loads(tool_call.function.arguments)  # Decodifica os argumentos da função do JSON
        function_response = function_to_call( # Chama a função e passa os argumentos extraidos
            local=function_args.get("local"),
            unidade=function_args.get("unidade"),
        )
        mensagens.append(      # Adiciona a resposta da função a conversa
            {
                "tool_call_id": tool_call.id,    # O id
                "role": "tool",     # Role é a ferramenta
                "name": function_name,     # O nome
                "content": function_response,   # E a resposta fornecida pelo modelo
            }
        )
        
    segunda_resposta = client.chat.completions.create(    # Capturando a resposta final do modelo, depois de utilizar a funcao
        model="gpt-3.5-turbo-0125",   
        messages=mensagens,
    )

mensagem_resp = segunda_resposta.choices[0].message   # Imprimindo a resposta final do modelo
print(mensagem_resp.content)

A temperatura em São Paulo é de 32°C e em Porto Alegre é de 25°C.
